In [6]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import networkx as nx

import random
import math

from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [7]:
CITIES = pd.read_csv('cities/vanuatu.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
    
dist_df = pd.DataFrame(DIST_MATRIX, index=CITIES['name'], columns=CITIES['name'])

In [8]:
def tsp_cost(tsp):
    assert tsp[0] == tsp[-1] #check the return to te starting city
    assert set(tsp) == set(range(len(CITIES))) #check to visit each cities

    tot_cost = 0
    for c1, c2 in zip(tsp, tsp[1:]):
        tot_cost += DIST_MATRIX[c1, c2]
        
    return tot_cost


# Simulated Annealing

In [9]:
# Funzione per generare un vicino (mutazione con scambio di due città)
def generate_neighbor(tour):
    new_tour = tour[:]
    i, j = random.sample(range(1, len(tour) - 1), 2)  # Escludiamo il primo e l'ultimo per mantenere l'inizio e la fine uguali
    new_tour[i], new_tour[j] = new_tour[j], new_tour[i]
    return new_tour

# Algoritmo di Simulated Annealing
def simulated_annealing(dist_matrix, initial_temp=10000, cooling_rate=0.995, temp_min=1):
    # Inizializza un tour casuale che inizia e finisce nella stessa città
    num_cities = dist_matrix.shape[0]
    current_tour = list(range(num_cities))
    random.shuffle(current_tour)
    current_tour.append(current_tour[0])  # Assicurati che il tour ritorni alla città di partenza
    
    # Calcola il costo del tour iniziale
    current_cost = tsp_cost(current_tour)
    best_tour = current_tour[:]
    best_cost = current_cost

    temp = initial_temp
    
    while temp > temp_min:
        # Genera un tour vicino al tour corrente
        neighbor_tour = generate_neighbor(current_tour)
        neighbor_cost = tsp_cost(neighbor_tour)
        
        # Calcola la differenza di costo tra il tour attuale e il vicino
        delta_cost = neighbor_cost - current_cost

        # Accetta il nuovo tour se è migliore, o con una probabilità decrescente se è peggiore
        if delta_cost < 0 or random.random() < math.exp(-delta_cost / temp):
            current_tour, current_cost = neighbor_tour, neighbor_cost
            
            # Aggiorna la migliore soluzione trovata
            if current_cost < best_cost:
                best_tour, best_cost = current_tour, current_cost
                print(f"Nuova miglior soluzione: costo = {best_cost}")

        # Riduci la temperatura
        temp *= cooling_rate
    
    return best_tour, best_cost

# Esegui l'algoritmo di Simulated Annealing
best_tour, best_distance = simulated_annealing(DIST_MATRIX)
print("Best tour found:", best_tour)
print("Distance of best tour:", best_distance)

Nuova miglior soluzione: costo = 1627.878003117094
Nuova miglior soluzione: costo = 1534.702855357689
Nuova miglior soluzione: costo = 1533.6561222633416
Nuova miglior soluzione: costo = 1498.3399039540736
Nuova miglior soluzione: costo = 1493.4287201697678
Nuova miglior soluzione: costo = 1397.6399891301412
Nuova miglior soluzione: costo = 1353.081827808743
Nuova miglior soluzione: costo = 1350.456140257617
Nuova miglior soluzione: costo = 1345.5449564733115
Nuova miglior soluzione: costo = 1345.5449564733112
Best tour found: [5, 6, 2, 0, 7, 1, 4, 3, 5]
Distance of best tour: 1345.5449564733112


# Hybrid Implementation

In [10]:
# Funzione 2-opt per miglioramento locale
def two_opt(tour):
    best = tour[:]
    improved = True
    while improved:
        improved = False
        for i in range(1, len(tour) - 2):
            for j in range(i + 1, len(tour) - 1):
                if DIST_MATRIX[best[i - 1], best[i]] + DIST_MATRIX[best[j], best[j + 1]] > DIST_MATRIX[best[i - 1], best[j]] + DIST_MATRIX[best[i], best[j + 1]]:
                    best[i:j+1] = best[i:j+1][::-1]
                    improved = True
    return best

# Scramble mutation
def scramble_mutation(tour):
    new_tour = tour[:]
    i, j = sorted(random.sample(range(1, len(tour) - 1), 2))
    scrambled_part = new_tour[i:j]
    random.shuffle(scrambled_part)
    new_tour[i:j] = scrambled_part
    return new_tour

# Nearest Neighbor tour per generare la popolazione iniziale
def nearest_neighbor_tour(start_city):
    unvisited = set(range(len(CITIES)))
    unvisited.remove(start_city)
    tour = [start_city]
    current_city = start_city
    while unvisited:
        next_city = min(unvisited, key=lambda city: DIST_MATRIX[current_city, city])
        unvisited.remove(next_city)
        tour.append(next_city)
        current_city = next_city
    tour.append(start_city)
    return tour

# Crea la popolazione iniziale
population = [two_opt(nearest_neighbor_tour(i)) for i in range(len(CITIES))]

# Funzione di evoluzione
def evolve_population(population, generations=100):
    best_solution = min(population, key=tsp_cost)
    best_cost = tsp_cost(best_solution)
    
    for generation in range(generations):
        new_population = []

        # Elitismo: mantieni il migliore
        new_population.append(best_solution)
        
        while len(new_population) < len(population):
            # Seleziona un individuo e applica la mutazione
            parent = random.choice(population)
            child = scramble_mutation(parent[:])
            
            # Miglioramento locale con 2-opt
            child = two_opt(child)
            
            new_population.append(child)
        
        # Valuta la nuova popolazione
        population = new_population
        current_best = min(population, key=tsp_cost)
        current_best_cost = tsp_cost(current_best)
        
        if current_best_cost < best_cost:
            best_solution, best_cost = current_best, current_best_cost
            print(f"Generazione {generation}, Miglior costo: {best_cost}")
    
    return best_solution, best_cost

# Esegui l'algoritmo genetico
best_tour, best_distance = evolve_population(population)
print("Best tour found:", best_tour)
print("Distance of best tour:", best_distance)

Best tour found: [0, 7, 1, 4, 3, 5, 6, 2, 0]
Distance of best tour: 1345.5449564733112
